In [1]:
import pandas as pd
from sklearn import linear_model
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
train = pd.read_csv("train_Madison.csv")
#train = train[:20]
test = pd.read_csv("test_Madison.csv")
names = train["name"]
stars = train["star"]    

In [16]:
ratingTotal = {}
ratingCount = {}
for i in range(0, len(names)):
    name = names[i]
    star = stars[i]
    if name not in ratingTotal:
        ratingTotal[name] = 0
        ratingCount[name] = 0
    ratingTotal[name] += star
    ratingCount[name]+=1
print(len(ratingTotal))
print(len(ratingCount))
ratingAverage = dict((k, float(ratingTotal[k])/ratingCount[k]) for k in ratingCount)
#Turns the predictor from rating to difference above or below mean rating for the restaurant
#for i in range(0, len(names)):
#    name = names[i]
#    diffScore = stars[i]-ratingAverage[name]
#    print(diffScore)
#    diffScores.append(diffScore)
train.head(5)

1150
1150


,Belleville,Black Earth,Columbus,Cottage Grove,Cross Plains,Dane,De Forest,DeForest,Deerfield,Deforest,...,Oregon,Paoli,Sauk City,Shorewood Hills,Stoughton,Sun Praiie,Sun Prairie,Verona,Waunakee,Windsor
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
positiveCount = []
negativeCount = []
positiveOverall = []
ratingArray = []
nwords = []
posnshort = []
negnlong = []
nchars = []
avgChars = []
posnegratio= []
numReviews = []
negativeplus = []
city=[]
positiveWords = ["gem", "highly", "incredible", "amazing", "die", "favorites", "wonderful", "perfect", "fantastic", "notch",
                 "favorite", "awesome", "outstanding", "yum", "delicious", "excellent", "perfectly", "loved", "helpful", "best", 
                 "savory", "cozy", "unique", "yummy", "glad", "homemade", "best", "love", "lovely", "always", "friendly", 
                 "reasonable", "beautiful", "recommended", "fashioned", "classic", "traditional", "great", "fresh", "definitely",
                 "healthy", "decent", "generous", "comfortable", "rich", "recommend", "local", "authentic", "flavorful",
                 "enjoyed", "pleased", "flavors", "super", "happy", "absolutely", "tasty", "enjoy", "fun", "reasonably", "plenty",
                 "attentive", "truly", "fancy", "wow", "must", "nicely", "every", "quick", "easily", "early", "craving", "well"]
negativeWords = ["disappointing", "weird", "issue", "none", "last", "problem", "dirty", "mediocre", "ok", "poor", "terrible",
                 "awful", "rude", "horrible", "worst", "overpriced", "needed", "barely", "sorry", "waited", "soggy", "waiting",
                 "bland", "forgot", "hope","unfortunately", "sad", "away", "bad", "loud", "overly", "greasy", "frozen", "dry",
                 "empty", "never", "less", "hard", "expensive", "tiny", "however", "wrong", "longer", "nothing", "average", 
                 "cold", "slow", "lack", "avoid", "forget", "left", "worst"]
#There are 2 'best's and 2 'worst's -> Accuracy improved
for i in range(0, len(names)):
    #print(i)
    #print(len(names))
    posC = 0
    negC = 0
    for word in positiveWords:
        wordCount = train[word][i]
        posC+=wordCount
    for negWord in negativeWords:
        negWordCount = train[negWord][i]
        negC+=negWordCount
    nword = np.array(train["nword"])[i]
    nchar = np.array(train["nchar"])[i]
    avgChars.append(nword/nchar)
    nwords.append(nword)
    nchars.append(nchar)
    positiveCount.append(posC)
    negativeCount.append(negC)
    posnegratio.append((posC)/(negC + 1))
    posnshort.append(posC/nword)
    negnlong.append(negC*nword)
    if posC>negC:
        positiveOverall.append(1)
    elif posC<negC:
        positiveOverall.append(-1)
    else:
        positiveOverall.append(0)
    ratingArray.append(ratingAverage[names[i]])
    numReviews.append(ratingCount[names[i]])

#predictors = np.array([nwords, positiveOverall, ratingArray, positiveCount, negativeCount, posRatio])
#posRatio = np.array(posRatio)
#print(posRatio)
predictors = np.array([numReviews, positiveOverall, ratingArray, positiveCount, negativeCount, posnshort, negnlong, avgChars, nwords, nchars, posnegratio])
predictors = predictors.T
reg = linear_model.LinearRegression()
reg.fit(predictors, train["star"])
acc_MLR= round(reg.score(predictors, train["star"])*100,2)
acc_MLR

# smallRatingAverage = dict((k, float(ratingTotal[k])/ratingCount[k]) for k in ratingCount if ratingCount[k]<3)
# smallTotal = 0
# for k in smallRatingAverage:
#     smallTotal+=float(smallRatingAverage[k])
# print("Small Total: ", len(smallRatingAverage))
# print("Small Average: ", smallTotal/len(smallRatingAverage))

# largeRatingAverage = dict((k, float(ratingTotal[k])/ratingCount[k]) for k in ratingCount if ratingCount[k]>=50)
# largeTotal = 0
# for k in largeRatingAverage:
#     largeTotal+=float(largeRatingAverage[k])
# print("Large Total: ", len(largeRatingAverage))    
# print("Large Average: ", largeTotal/len(largeRatingAverage))

#if negC>3:
#        negativeplus.append(-1)
#    else:
#        negativeplus.append(0)
#All predictors : 47.8
#numReviews, positiveOverall, ratingArray, positiveCount, negativeCount, posnshort, negnlong, avgChars, nwords, nchars, posnegratio
#Postive Overall : 35.03
#posnegratio: 19.29
#postiveCount,negativeCount : 32.41
#posnshort, negnlong : 23.92
#numReviews: 0.58
#nwords : 2.65
#nchars: 2.35
#avgChars: 2.75(nchars/nwords), 2.77(nwords/ nchars)
#nwords, nchars : 4.69
#ratingArray : 18.6
#positveOverall, postiveCount, negativeCount : 38.93
#positveOverall, postiveCount, negativeCount, posnshort, negnlong : 41.08  +posnegratio:41.1
#+nChar: 41.44(better than avgChar) #+nwords :41.5

ValueError: setting an array element with a sequence.

In [14]:
testNames = test["name"]
testIDs = test["Id"]
testExpected = []
for i in range (0, len(testNames)):
    name = testNames[i]
    posC = 0
    negC = 0
    posOverall = 0
    if name in ratingAverage:
        rating = ratingAverage[name]
    else:
        rating = 3.758
        
    for word in positiveWords:
        wordCount = test[word][i]
        posC+=wordCount
    for word in negativeWords:
        wordCount = test[word][i]
        negC+=wordCount 
    if posC > negC:
        posOverall = 1
    elif posC < negC:
        posOverall = -1
    else:
        posOverall = 0
    
    nword = np.array(test["nword"])[i]
    nchar = np.array(test["nchar"])[i]
    avgChar = nchar/nword
    #numReviews = ratingCount[name]
    posnshort = posC/nword
    negnlong = negC*nword
    posnegratio = ((posC)/(negC + 1))
    #predictors = np.array([[posC, negC, rating]])
    predictors = np.array([[posOverall, rating, posC, negC, posnshort, negnlong, avgChar, nword, nchar, posnegratio]])
    prediction = reg.predict(predictors)
    #prediction = prediction+rating
    testExpected.append(prediction[0])
    #print(prediction)
submitDF = pd.DataFrame({'Id':testIDs, 'Expected':testExpected})
submitDF.to_csv("submission01_d.csv", sep=',', index=False)

NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.